# [Weekly Mission] w4_RecSys_Trend_Implement **GCL4SR**


## 문제정의
저희는 RecSys Trend 강의에서 다음과 같은 내용을 배웠습니다!

**Graph-based RecSys**
- 추천에서 사용되는 데이터를 유저-아이템 상호작용 그래프로 구성하여 GCN 구조를 사용한 추천 모델을 설계할 수 있다.
- 특히, SGL (SIGIR 21)은 Graph Contrastive Learning을 사용하여 추천의 성능을 향상시켰다.

**Sequential RecSys**
- 유저가 상호작용한 아이템은 그 순서 정보가 존재하며, 이를 활용하면 더 좋은 추천 결과를 얻을 수 있다.
- 이를 위해, Transformer와 같은 모델을 사용해서 Sequential 정보를 사용하도록 모델을 설계한다.

Graph-enhanced Sequential Recommendation는 위 두 가지 접근법을 적절히 융합하여,
그래프로 표현되는 유저-아이템 사이의 연결관계 정보와, 개별 유저가 상호작용한 아이템의 순서 정보를 동시에 활용할 수 있도록 하는 방법론입니다. 이번 위클리 미션에서는 Graph-enhanced Sequential Recommendation 모델 중 하나인 **GCL4SR** (IJCAI 22) 모델의 일부를 구현해볼 것이며, 이를 통해 Graph-based 모델의 강점을 어떻게 Sequential로 녹일 수 있는지, 그리고 추천시스템 학습 및 평가에서 사용되는 전반적인 코드 구조를 이해해보는 것이 목표입니다!

**GCL4SR**에 대한 세부내용은 보충자료와 논문 참고 부탁드립니다.

---

논문 : Enhancing Sequential Recommendation with Graph Contrastive Learning (IJCAI 22) [link](https://www.ijcai.org/proceedings/2022/0333.pdf)

## 라이브러리 임포트
```bash
torch >= 2.7
torch-geometric >= 2.6.1
torch_sparse >= 0.6.18
torch_scatter >= 2.1.2
numpy >= 2.2.6
```

In [8]:
import os
import random
import pickle

import numpy as np
from scipy.sparse import csr_matrix

import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from torch_geometric.data import Data

from torch import nn, Tensor
from torch.autograd import Variable
from torch.nn.init import xavier_normal_, constant_
from torch_geometric.loader import NeighborSampler
from torch_geometric.nn import SAGEConv, GCNConv
from torch.nn import Module
import torch.nn.functional as F
import numpy as np

from tqdm import tqdm

import math
from numpy import lexsort

from typing import List, Tuple, Dict, Set

## 환경 설정
데이터셋은 첨부 파일을 확인해주세요.

### 데이터셋 경로 지정

In [9]:
# Raw data file
data_file  = './dataset/home.txt'

# Training sequence file and Weight Item Transition Graph (WITG) file
train_sequence_file = './dataset/all_train_seq.txt'
witg_file = './dataset/witg.pt'

# Splited dataset files
train_file = './dataset/train.pkl'
valid_file = './dataset/valid.pkl'
test_file  = './dataset/test.pkl'

# Model checkpoint file
output_dir = 'output/'
checkpoint_file = output_dir + 'checkpoint.pth'

In [10]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f'{output_dir} created')

### GPU 사용 여부 확인

In [11]:
if torch.cuda.is_available():
    print('Using GPU')
else:
    print('Using CPU')

Using CPU


### 재현성을 위한 시드 설정

In [12]:
# 재현성을 위한 시드 설정
seed = 2026

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True

### 기타 유틸리티 함수 및 클래스 선언 (수정 금지)

In [13]:
def generate_rating_matrix_train(user_seq, num_users, num_items):
    row = []
    col = []
    data = []
    for user_id, item_list in enumerate(user_seq):
        for item in item_list[:-2]: #
            row.append(user_id)
            col.append(item)
            data.append(1)

    row = np.array(row)
    col = np.array(col)
    data = np.array(data)
    rating_matrix = csr_matrix((data, (row, col)), shape=(num_users, num_items))

    return rating_matrix

In [14]:
def generate_rating_matrix_test(user_seq, num_users, num_items):
    row = []
    col = []
    data = []
    for user_id, item_list in enumerate(user_seq):
        for item in item_list[:-1]: #
            row.append(user_id)
            col.append(item)
            data.append(1)
            
    row = np.array(row)
    col = np.array(col)
    data = np.array(data)
    rating_matrix = csr_matrix((data, (row, col)), shape=(num_users, num_items))

    return rating_matrix

In [15]:
def get_matrix_and_num(data_file):
    lines = open(data_file).readlines()
    user_seq = []
    item_set = set()
    for line in lines:
        user, items = line.strip().split(' ', 1)
        items = items.split(',')
        items = [int(item) for item in items]
        user_seq.append(items)
        item_set = item_set | set(items)
    max_item = max(item_set)

    num_users = len(lines)
    num_items = max_item + 1

    train_matrix = generate_rating_matrix_train(user_seq, num_users, num_items)
    test_matrix = generate_rating_matrix_test(user_seq, num_users, num_items)
    return num_users, num_items, train_matrix, test_matrix

In [16]:
class GCL4SRData(Dataset):
    def __init__(self, data, max_seq_length):
        self.max_len = max_seq_length
        self.data = data
        self.uid_list = data[0]
        self.part_sequence = data[1]
        self.part_sequence_target = data[2]
        self.part_sequence_length = data[3]
        self.length = len(data[0])

    def __getitem__(self, index):
        input_ids = self.part_sequence[index]
        target_pos = self.part_sequence_target[index]
        user_id = self.uid_list[index]

        pad_len = self.max_len - len(input_ids)
        input_ids = [0] * pad_len + input_ids

        input_ids = input_ids[-self.max_len:]

        cur_tensors = (
            torch.tensor(user_id, dtype=torch.long),
            torch.tensor(input_ids, dtype=torch.long),
            torch.tensor(target_pos, dtype=torch.long),
        )
        return cur_tensors

    def __len__(self):
        return self.length

In [17]:
def grade_witg(student_graph: Data, answer_graph: Data):
    print("\n--- WITG SANITY CHECK ---")
    
    if student_graph.num_nodes == answer_graph.num_nodes:
        print(f"Number of nodes: Match ({student_graph.num_nodes})")
    else:
        print(f"Number of nodes: Mismatch (Student: {student_graph.num_nodes}, Answer: {answer_graph.num_nodes})")

    try:
        # Sort student graph edges
        student_perm = lexsort(keys=(student_graph.edge_index[1].cpu().numpy(), student_graph.edge_index[0].cpu().numpy()))
        student_sorted_edges = student_graph.edge_index[:, student_perm]
        student_sorted_attrs = student_graph.edge_attr[student_perm]

        # Sort answer graph edges
        answer_perm = lexsort(keys=(answer_graph.edge_index[1].cpu().numpy(), answer_graph.edge_index[0].cpu().numpy()))
        answer_sorted_edges = answer_graph.edge_index[:, answer_perm]
        answer_sorted_attrs = answer_graph.edge_attr[answer_perm]

        # Compare edge structures
        if torch.equal(student_sorted_edges, answer_sorted_edges):
            print("Edge connectivity structure: Match")
        else:
            print("Edge connectivity structure: Mismatch")

        # Compare edge attributes (weights)
        if torch.allclose(student_sorted_attrs, answer_sorted_attrs):
            print("Edge weights: Match")
        else:
            print("Edge weights: Mismatch")
            
    except Exception as e:
        print(f"Error during edge comparison: {e}")

In [18]:
def grade_loss(student_loss):
    answer_loss = (9.9495, 15.3257, 0.2743)
    ans_main_loss, ans_gcl_loss, ans_mmd_loss = answer_loss
    
    try:
        stu_main_loss, stu_gcl_loss, stu_mmd_loss = student_loss
        float(stu_main_loss), float(stu_gcl_loss), float(stu_mmd_loss)
    except (TypeError, ValueError):
        print("\n--- LOSS SANITY CHECK FAILED ---")
        print("Error: student_loss must contain three numerical values.")
        print(f"Received input: {student_loss}")
        return

    print("\n--- LOSS SANITY CHECK ---")
    print("Loss 값은 GPU 하드웨어에 따라 약간에 오차가 발생할 수 있습니다. 이는 감안하여 채점할 것이니 걱정하지 않으셔도 됩니다.")
    print("이 함수는 구현이 잘되었는지 간단히 확인하는 용도로 제공하는 함수입니다!")

    TOLERANCE = 1e-3

    if math.isclose(ans_main_loss, stu_main_loss, rel_tol=TOLERANCE):
        print(f"Main Loss: Match (Answer: {ans_main_loss:.4f}, Student: {stu_main_loss:.4f})")
    else:
        print(f"Main Loss: Mismatch (Answer: {ans_main_loss:.4f}, Student: {stu_main_loss:.4f})")

    if math.isclose(ans_gcl_loss, stu_gcl_loss, rel_tol=TOLERANCE):
        print(f"GCL Loss:  Match (Answer: {ans_gcl_loss:.4f}, Student: {stu_gcl_loss:.4f})")
    else:
        print(f"GCL Loss:  Mismatch (Answer: {ans_gcl_loss:.4f}, Student: {stu_gcl_loss:.4f})")

    if math.isclose(ans_mmd_loss, stu_mmd_loss, rel_tol=TOLERANCE):
        print(f"MMD Loss:  Match (Answer: {ans_mmd_loss:.4f}, Student: {stu_mmd_loss:.4f})")
    else:
        print(f"MMD Loss:  Mismatch (Answer: {ans_mmd_loss:.4f}, Student: {stu_mmd_loss:.4f})")

In [19]:
def grade_eval(recall_10: float, recall_20: float, ndcg_10: float, ndcg_20: float):
    answer_metrics = {
        "Recall@10": 0.0082,
        "Recall@20": 0.0143,
        "NDCG@10":   0.0040,
        "NDCG@20":   0.0056
    }

    student_metrics = {
        "Recall@10": recall_10,
        "Recall@20": recall_20,
        "NDCG@10":   ndcg_10,
        "NDCG@20":   ndcg_20
    }
    print("\n--- EVALUATION METRICS SANITY CHECK ---")
    print("마찬가지로 Metric 값도 GPU 하드웨어에 따라 약간에 오차가 발생할 수 있습니다. 이는 감안하여 채점할 것이니 걱정하지 않으셔도 됩니다.")
    print("이 함수는 구현이 잘되었는지 간단히 확인하는 용도로 제공하는 함수입니다!")

    TOLERANCE = 1e-3

    for metric_name, ans_value in answer_metrics.items():
        stu_value = student_metrics[metric_name]
        display_name = f"{metric_name}:".ljust(12)
        if math.isclose(ans_value, stu_value, rel_tol=TOLERANCE):
            print(f"{display_name} Match   (Answer: {ans_value:.4f}, Student: {stu_value:.4f})")
        else:
            print(f"{display_name} Mismatch (Answer: {ans_value:.4f}, Student: {stu_value:.4f})")


## 데이터 불러오기

In [20]:
user_num, item_num, train_matrix, test_matrix = get_matrix_and_num(data_file)
print(f'num_users: {user_num}, num_items: {item_num}')

num_users: 66519, num_items: 28238


In [21]:
train_data = pickle.load(open(train_file, 'rb'))
valid_data = pickle.load(open(valid_file, 'rb'))
test_data  = pickle.load(open(test_file , 'rb'))

## TODO1: Weighted Item Transition Graph (WITG) 구성하기  

논문에 제시된 WITG 구성 방법을 구현하는 함수 `build_weighted_item_transition_graph`를 완성해주세요.  
이를 위해서, `read_item_sequences`, `convert_to_pyg_data` 함수의 입출력을 살펴보아야 합니다.  
WITG 구성 방법은 수식으로 보면 아래와 같습니다.

- $k= 1, 2, 3$ : window size  
- $1/k$ : 노드 $v_t$가 노드 $v_{t+k}$에 영향을 미치는 점수  

$$
w(v_t, v_{t+k}) = \begin{cases}
1/k & \text{if }v_t, v_{t+k}\text{를 연결하는 엣지가 WITG에 없는 경우} \\
w(v_t, v_{t+k}) + 1/k & \text{if }v_t, v_{t+k}\text{를 연결하는 엣지가 이미 WITG에 있는 경우}
\end{cases}
$$

In [22]:
def read_item_sequences(file_path: str) -> Tuple[List[List[int]], int]:
    """
    사용자별 아이템 시퀀스 파일을 읽어 파싱합니다.

    Args:
        file_path (str): 'user_id item1,item2,...' 형식의 텍스트 파일 경로.

    Returns:
        Tuple[List[List[int]], int]: 
            - 모든 사용자의 아이템 시퀀스 목록 (e.g., [[1,2,3], [4,5]])
            - (가장 큰 아이템 ID + 1)을 의미하는 전체 노드 개수
    """
    user_sequences = []
    all_items = set()

    with open(file_path, 'r') as f:
        for line in f:
            # "user_id item1,item2,..." -> ["user_id", "item1,item2,..."]
            _, items_str = line.strip().split(' ', 1)
            items = [int(item) for item in items_str.split(',')]
            
            user_sequences.append(items)
            all_items.update(items)

    num_nodes = max(all_items) + 1 if all_items else 0
    return user_sequences, num_nodes

In [23]:
def convert_to_pyg_data(adjacency_list: List[Dict[int, float]], num_nodes: int) -> Data:
    """
    인접 리스트 형태의 그래프를 PyTorch Geometric(PyG)의 Data 객체로 변환합니다.

    Args:
        adjacency_list (List[Dict[int, float]]): 
            - 그래프의 인접 리스트. 
            - e.g., adj[source_node] = {target_node_1: weight_1, ...}
        num_nodes (int): 그래프의 전체 노드 개수.

    Returns:
        Data: PyG 모델에서 사용할 수 있는 그래프 데이터 객체.
    """
    edge_list = []
    weight_list = []

    # 각 노드에 대해, 이웃 노드들을 가중치(weight)가 높은 순으로 정렬
    for source_node, neighbors in enumerate(adjacency_list):
        # neighbors.items() -> [(target_node, weight), ...]
        sorted_neighbors = sorted(neighbors.items(), key=lambda item: item[1], reverse=True)
        
        for target_node, weight in sorted_neighbors:
            edge_list.append([source_node, target_node])
            weight_list.append(weight)

    # PyG가 요구하는 텐서 형태로 변환
    # edge_index: [2, num_edges] 형태의 LongTensor
    edge_index = torch.tensor(edge_list, dtype=torch.long).t().contiguous()
    
    # edge_attr: [num_edges, 1] 형태의 FloatTensor
    edge_attr = torch.tensor(weight_list, dtype=torch.float).view(-1, 1)
    
    # node_features: [num_nodes, 1] 형태, 각 노드의 ID를 특징으로 사용
    node_features = torch.arange(num_nodes, dtype=torch.long).view(-1, 1)

    graph_data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_attr)
    return graph_data

In [24]:
def build_weighted_item_transition_graph(train_sequence_file: str) -> Data:
    """
    아이템 시퀀스 데이터로부터 가중치가 있는 아이템 관계 그래프(WITG)를 생성합니다.

    Args:
        train_sequence_file (str): 학습 데이터로 사용할 시퀀스 파일 경로.

    Returns:
        Data: 완성된 가중치 그래프의 PyG Data 객체.
    """
    user_sequences, num_nodes = read_item_sequences(train_sequence_file)
    
    # 인접 리스트: adj[i]는 아이템 i와 연결된 (이웃 아이템, 가중치) 딕셔너리
    adjacency_list: List[Dict[int, float]] = [dict() for _ in range(num_nodes)]

    # 모든 사용자의 행동 시퀀스를 순회
    for sequence in user_sequences:
        # 한 시퀀스 내에서 아이템 쌍을 추출 (윈도우 사이즈: 1, 2, 3)
        for window_size in range(1, 4):
            for i in range(len(sequence) - window_size):
    # ======================= TODO: 이 부분을 구현하세요 ========================= #
                source_item = sequence[i]
                target_item = sequence[i + window_size]
                
                # 가중치는 거리(window_size)의 역수: 가까울수록 높은 가중치
                weight = 1.0 / window_size
                
                # 양방향(undirected)으로 엣지 가중치를 더해줌
                # .get(key, 0.0)은 키가 없으면 0.0을 반환하여 KeyError 방지
                adjacency_list[source_item][target_item] = adjacency_list[source_item].get(target_item, 0.0) + weight
                adjacency_list[target_item][source_item] = adjacency_list[target_item].get(source_item, 0.0) + weight
    # =========================================================================== #
    
    # 완성된 인접 리스트를 PyG 데이터 객체로 변환
    graph_data: Data = convert_to_pyg_data(adjacency_list, num_nodes)
    return graph_data

In [25]:
if os.path.exists(witg_file):
    print(f'Loading WITG from {witg_file}')
    global_graph = torch.load(witg_file, weights_only=False)
else:
    print(f'Building WITG and saving to {witg_file}')
    global_graph = build_weighted_item_transition_graph(train_sequence_file=train_sequence_file)
    torch.save(global_graph, witg_file)


# 만든 WITG를 정답 WITG와 비교합니다.
answer_witg = torch.load('./dataset/answer_witg.pt', weights_only=False)
print(f"WITG structure       : {global_graph}")
print(f"Answer WITG structure: {answer_witg}")
grade_witg(student_graph=global_graph, answer_graph=answer_witg)

Loading WITG from ./dataset/witg.pt
WITG structure       : Data(x=[28238, 1], edge_index=[2, 1617638], edge_attr=[1617638, 1])
Answer WITG structure: Data(x=[28238, 1], edge_index=[2, 1617638], edge_attr=[1617638, 1])

--- WITG SANITY CHECK ---
Number of nodes: Match (28238)
Edge connectivity structure: Match
Edge weights: Match


## 모델 학습

In [26]:
# 위클리 미션 임을 감안하여, epoch는 1로 설정하여 테스트합니다.
# 결과 비교를 위해서, 하이퍼 파라미터는 조정하지 않아야 합니다!
epochs = 1  
batch_size = 2048
hidden_size = 64
max_seq_length = 50
num_hidden_layers = 2
num_attention_heads = 2
sample_size = [20, 20]
lam1 = 1.0
lam2 = 0.1

### [TODO2] Metric 구현하기
추천 시스템에서 가장 많이 사용되는, Recall@k와 NDCG@k를 구현해주세요. 두 metric의 수식은 아래과 같습니다.  

$Recall@k = \frac{\text{Number of recommended items in top-k that are relevant}}{\text{Total number of relevant items}}$

$NDCG@k = \frac{DCG@k}{IDCG@k}$

$DCG@k = \sum_{i=1}^{k} \frac{rel_i}{\log_2(i+1)}$

$rel_i$는 $i$번째 위치에 있는 아이템의 **관련성(relevance)**입니다.  
일반적으로 해당 아이템이 **정답(target) 아이템**인 경우 **1**이고, 그렇지 않은 경우 **0**입니다.

In [27]:
def recall_at_k(actual: List[list], predicted: List[list], topk: int) -> float:
    """
    추천 시스템의 평균 Recall@k를 계산합니다.

    Recall@k는 사용자가 실제로 관심 있었던 전체 아이템 중에서,
    모델이 추천한 상위 K개의 아이템이 얼마나 포함하고 있는지를 나타내는 지표입니다.

    Args:
        actual (List[list]): 사용자별 실제 정답 아이템 목록.
                             e.g., [[1, 2, 3], [4, 5]]
        predicted (List[list]): 모델이 추천한 사용자별 아이템 목록 (점수 순으로 정렬됨).
                                e.g., [[1, 5, 2], [4, 8, 9]]
        topk (int): 평가에 사용할 상위 K개의 추천 개수.

    Returns:
        float: 모든 사용자의 Recall@k 점수를 산술 평균한 값.
    """
    total_recall_score = 0.0
    num_evaluated_users = 0  # 평가가 가능한 사용자(정답 아이템이 있는) 수

    # ============================== TODO: 이 부분을 구현하세요 ================================ #
    # 모든 사용자에 대해 반복
    for user_id in range(len(predicted)):
        ground_truth_items: Set[int] = set(actual[user_id])
        
        # 평가를 위해선 사용자의 정답 아이템이 최소 1개 이상이어야 함
        if not ground_truth_items:
            continue
            
        num_evaluated_users += 1
        
        # 모델이 추천한 상위 topk개의 아이템 목록
        recommended_items: Set[int] = set(predicted[user_id][:topk])
        
        # 추천된 아이템 중 정답 아이템의 개수 (적중 개수)
        num_hits = len(ground_truth_items.intersection(recommended_items))
        
        # 현재 사용자의 Recall@k 점수 계산: (적중 개수) / (전체 정답 개수)
        recall_score = num_hits / len(ground_truth_items)
        total_recall_score += recall_score
    # ======================================================================================= #
    
    # 평가된 모든 사용자의 점수를 평균내어 최종 점수 계산
    mean_recall = total_recall_score / num_evaluated_users if num_evaluated_users > 0 else 0.0
    return mean_recall

In [28]:
def _calculate_idcg(k: int) -> float:
    """
    IDCG@k (Ideal Discounted Cumulative Gain) 값을 계산하는 헬퍼(helper) 함수.
    가장 이상적인 추천, 즉 상위 k개의 아이템이 모두 정답이라고 가정했을 때의 DCG 점수입니다.

    Args:
        k (int): IDCG를 계산할 목록의 길이.

    Returns:
        float: IDCG@k 점수.
    """
    ideal_dcg = sum([1.0 / math.log2(rank + 1) for rank in range(1, k + 1)])
    return ideal_dcg

In [29]:
def ndcg_at_k(actual: List[list], predicted: List[list], topk: int) -> float:
    """
    추천 시스템의 평균 NDCG@k를 계산합니다.

    NDCG@k는 추천 결과의 순서까지 고려하는 정밀한 평가 지표입니다.
    정답 아이템이 추천 목록의 앞쪽에 있을수록 높은 점수를 받습니다.

    Args:
        actual (List[list]): 사용자별 실제 정답 아이템 목록.
        predicted (List[list]): 모델이 추천한 사용자별 아이템 목록 (점수 순으로 정렬됨).
        topk (int): 평가에 사용할 상위 K개의 추천 개수.

    Returns:
        float: 모든 사용자의 NDCG@k 점수를 산술 평균한 값.
    """
    total_ndcg_score = 0.0
    num_evaluated_users = 0  # 평가가 가능한 사용자 수
    # ============================== TODO: 이 부분을 구현하세요 ================================ #
    for user_id in range(len(predicted)):
        ground_truth_items: Set[int] = set(actual[user_id])
        
        if not ground_truth_items:
            continue
            
        num_evaluated_users += 1
        
        # 1. DCG@k (Discounted Cumulative Gain) 계산
        dcg_score = 0.0
        # 추천 순위(rank)는 1부터 시작
        for rank, item_id in enumerate(predicted[user_id][:topk], 1):
            # 추천한 아이템이 정답 목록에 있다면, 순위에 따라 점수를 할인하여 더함
            if item_id in ground_truth_items:
                dcg_score += 1.0 / math.log2(rank + 1)
        
        # 2. IDCG@k (Ideal DCG) 계산 (가장 이상적인 추천일 경우의 DCG 값)
        idcg_score = _calculate_idcg(min(topk, len(ground_truth_items)))
        
        # 3. NDCG@k 계산 및 누적 (0으로 나누는 경우 방지)
        if idcg_score > 0:
            ndcg_score = dcg_score / idcg_score
            total_ndcg_score += ndcg_score
    # ======================================================================================= #
    mean_ndcg = total_ndcg_score / num_evaluated_users if num_evaluated_users > 0 else 0.0
    return mean_ndcg

### Trainer 선언

In [30]:
class Trainer:
    def __init__(self, model, optimizer, sample_size, hidden_size, train_matrix):
        self.model = model
        self.optimizer = optimizer
        self.sample_size = sample_size
        self.hidden_size = hidden_size
        self.train_matrix = train_matrix
        self.model.to(self.model.device)

    def get_scores(self, answers, pred_list):
        recall_10 = recall_at_k(answers, pred_list, 10)
        recall_20 = recall_at_k(answers, pred_list, 20)
        ndcg_10 = ndcg_at_k(answers, pred_list, 10)
        ndcg_20 = ndcg_at_k(answers, pred_list, 20)
        return recall_10, recall_20, ndcg_10, ndcg_20

    def predict(self, seq_out):
        test_item_emb = self.model.item_embeddings.weight
        rating_pred = torch.matmul(seq_out, test_item_emb.transpose(0, 1))
        return rating_pred

    def train_step(self, epoch, train_dataloader):
        self.model.train()
        main_loss_sum = 0.0
        gcl_loss_sum = 0.0
        mmd_loss_sum = 0.0

        for _, batch in tqdm(enumerate(train_dataloader), desc=f"Epoch {epoch}", total=len(train_dataloader)):
            batch = tuple(t.to(self.model.device) for t in batch)
            loss, main_loss, gcl_loss, mmd_loss = self.model.calculate_loss(batch)
            
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()

            main_loss_sum += main_loss.item()
            gcl_loss_sum += gcl_loss.item()
            mmd_loss_sum += mmd_loss.item()

        main_loss_avg = main_loss_sum / len(train_dataloader)
        gcl_loss_avg = gcl_loss_sum / len(train_dataloader)
        mmd_loss_avg = mmd_loss_sum / len(train_dataloader)

        return main_loss_avg, gcl_loss_avg, mmd_loss_avg

    def eval_step(self, dataloader, test_matrix):
        self.model.eval()
        pred_list = None
        answer_list = None

        for i, batch in tqdm(enumerate(dataloader), desc="Evaluate", total=len(dataloader)):
            batch = tuple(t.to(self.model.device) for t in batch)
            user_ids = batch[0]
            answers = batch[2]
            recommend_output = self.model.eval_stage(batch)
            answers = answers.view(-1, 1)

            rating_pred = self.predict(recommend_output)
            rating_pred = rating_pred.cpu().data.numpy().copy()
            batch_user_index = user_ids.cpu().numpy()
            rating_pred[test_matrix[batch_user_index].toarray() > 0] = 0
            ind = np.argpartition(rating_pred, -20)[:, -20:]
            arr_ind = rating_pred[np.arange(len(rating_pred))[:, None], ind]
            arr_ind_argsort = np.argsort(arr_ind)[np.arange(len(rating_pred)), ::-1]
            batch_pred_list = ind[np.arange(len(rating_pred))[:, None], arr_ind_argsort]

            if i == 0:
                pred_list = batch_pred_list
                answer_list = answers.cpu().data.numpy()
            else:
                pred_list = np.append(pred_list, batch_pred_list, axis=0)
                answer_list = np.append(answer_list, answers.cpu().data.numpy(), axis=0)

        recall_10, recall_20, ndcg_10, ndcg_20 = self.get_scores(answer_list, pred_list)
        return recall_10, recall_20, ndcg_10, ndcg_20

### [TODO3] Model 구현하기

여기서는 논문에서 나온 Maximum Mean Discrepancy (MMD) loss를 구현할 것입니다.  
가우시안 커널을 정의한 `gaussian_kernel` 함수의 입출력을 면밀히 살펴보신 다음,  
아래의 MMD loss의 정의에 따라서 `MMD_Loss`를 구현해주시면 됩니다.  



In [31]:
class GNN_Encoder(Module):
    def __init__(self, hidden_size, sample_size, gnn_dropout_prob):
        super(GNN_Encoder, self).__init__()
        self.hidden_size = hidden_size
        in_channels = hidden_channels = self.hidden_size
        self.num_layers = len(sample_size)
        self.dropout = nn.Dropout(gnn_dropout_prob)
        self.gcn = GCNConv(self.hidden_size, self.hidden_size)
        self.convs = nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels, normalize=True))
        for i in range(self.num_layers - 1):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels, normalize=True))


    def forward(self, x, adjs, attr):
        xs = []
        x_all = x
        if self.num_layers > 1:
            for i, (edge_index, e_id, size) in enumerate(adjs):
                weight = attr[e_id].view(-1).type(torch.float)

                x = x_all
                if len(list(x.shape)) < 2:
                    x = x.unsqueeze(0)
                x = self.gcn(x, edge_index, weight)
                # sage
                x_target = x[:size[1]]  # Target nodes are always placed first.
                x = self.convs[i]((x, x_target), edge_index)
                if i != self.num_layers - 1:
                    x = F.relu(x)
                    x = self.dropout(x)
        else:
            edge_index, e_id, size = adjs.edge_index, adjs.e_id, adjs.size
            x = x_all
            x = self.dropout(x)
            weight = attr[e_id].view(-1).type(torch.float)
            if len(list(x.shape)) < 2:
                x = x.unsqueeze(0)
            x = self.gcn(x, edge_index, weight)
            x_target = x[:size[1]]  # Target nodes are always placed first.
            x = self.convs[-1]((x, x_target), edge_index)
        xs.append(x)
        return torch.cat(xs, 0)

In [32]:
class GCL4SR(nn.Module):
    def __init__(self, user_num, item_num, hidden_size, max_seq_length, num_attention_heads, global_graph, num_hidden_layers, lam1, lam2, sample_size):
        super(GCL4SR, self).__init__()

        self.user_num = user_num
        self.item_num = item_num
        self.hidden_size = hidden_size
        self.sample_size = sample_size
        self.max_seq_length = max_seq_length
        self.lam1 = lam1
        self.lam2 = lam2

        if torch.cuda.is_available():
            self.device = torch.device('cuda')
        else:
            self.device = torch.device('cpu')
        
        self.global_graph = global_graph.to(self.device)
        self.global_gnn = GNN_Encoder(hidden_size, sample_size, gnn_dropout_prob=0.5)

        self.user_embeddings = nn.Embedding(user_num, hidden_size)
        self.item_embeddings = nn.Embedding(item_num, hidden_size, padding_idx=0)
        self.position_embeddings = nn.Embedding(max_seq_length, hidden_size)

        # sequence encoder
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=hidden_size,
                                                        nhead=num_attention_heads,
                                                        dim_feedforward=4 * hidden_size,
                                                        dropout=0.5,
                                                        activation='gelu',
                                                        batch_first=True)
        
        self.item_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_hidden_layers)

        # AttNet
        self.w_1 = nn.Parameter(torch.Tensor(2*hidden_size, hidden_size))
        self.w_2 = nn.Parameter(torch.Tensor(hidden_size, 1))
        self.linear_1 = nn.Linear(hidden_size, hidden_size)
        self.linear_2 = nn.Linear(hidden_size, hidden_size, bias=False)

        self.w_g = nn.Linear(hidden_size, 1)
        self.w_e = nn.Linear(hidden_size, 1)

        self.LayerNorm = nn.LayerNorm(hidden_size, eps=1e-12)
        self.dropout = nn.Dropout(0.5)
        self.linear_transform = nn.Linear(3*hidden_size, hidden_size, bias=False)
        self.gnndrop = nn.Dropout(0.5)

        self.criterion = nn.CrossEntropyLoss()
        self.apply(self._init_weights)

        # user-specific gating
        self.gate_item = Variable(torch.zeros(hidden_size, 1).type
                                  (torch.FloatTensor), requires_grad=True).to(self.device)
        self.gate_user = Variable(torch.zeros(hidden_size, max_seq_length).type
                                  (torch.FloatTensor), requires_grad=True).to(self.device)
        self.gate_item = torch.nn.init.xavier_uniform_(self.gate_item)
        self.gate_user = torch.nn.init.xavier_uniform_(self.gate_user)


    def _init_weights(self, module):
        """ Initialize the weights """
        stdv = 1.0 / math.sqrt(self.hidden_size)
        for weight in self.parameters():
            weight.data.uniform_(-stdv, stdv)
        if isinstance(module, nn.Embedding):
            xavier_normal_(module.weight.data)
        elif isinstance(module, nn.Linear):
            xavier_normal_(module.weight.data)
            if module.bias is not None:
                constant_(module.bias.data, 0)


    def gnn_encode(self, items):
        subgraph_loaders = NeighborSampler(self.global_graph.edge_index, node_idx=items, sizes=self.sample_size,
                                           shuffle=False,
                                           num_workers=0, batch_size=items.shape[0])
        g_adjs = []
        s_nodes = []
        for (b_size, node_idx, adjs) in subgraph_loaders:
            if type(adjs) == list:
                g_adjs = [adj.to(items.device) for adj in adjs]
            else:
                g_adjs = adjs.to(items.device)
            n_idxs = node_idx.to(items.device)
            s_nodes = self.item_embeddings(n_idxs).squeeze()
        attr = self.global_graph.edge_attr.to(items.device)
        g_hidden = self.global_gnn(s_nodes, g_adjs, attr)
        return g_hidden


    def final_att_net(self, seq_mask, hidden):
        batch_size = hidden.shape[0]
        lens = hidden.shape[1]
        pos_emb = self.position_embeddings.weight[:lens]
        pos_emb = pos_emb.unsqueeze(0).repeat(batch_size, 1, 1)
        seq_hidden = torch.sum(hidden * seq_mask, -2) / torch.sum(seq_mask, 1)
        seq_hidden = seq_hidden.unsqueeze(-2).repeat(1, lens, 1)
        item_hidden = torch.matmul(torch.cat([pos_emb, hidden], -1), self.w_1)
        item_hidden = torch.tanh(item_hidden)
        score = torch.sigmoid(self.linear_1(item_hidden) + self.linear_2(seq_hidden))
        att_score = torch.matmul(score, self.w_2)
        att_score_masked = att_score * seq_mask
        output = torch.sum(att_score_masked * hidden, 1)
        return output


    def generate_square_subsequent_mask(self, sz: int) -> Tensor:
        mask = torch.triu(torch.ones(sz, sz, dtype=torch.bool), diagonal=1)
        return mask


    def forward(self, data):
        user_ids = data[0]
        inputs = data[1]

        seq = inputs.flatten()
        seq_mask = (inputs == 0).float().unsqueeze(-1)
        seq_mask = 1.0 - seq_mask

        seq_hidden_global_a = self.gnn_encode(seq).view(-1, self.max_seq_length, self.hidden_size)
        seq_hidden_global_b = self.gnn_encode(seq).view(-1, self.max_seq_length, self.hidden_size)

        key_padding_mask = (inputs == 0)
        attn_mask = self.generate_square_subsequent_mask(self.max_seq_length).to(inputs.device)
        seq_hidden_local = self.item_embeddings(inputs)
        seq_hidden_local = self.LayerNorm(seq_hidden_local)
        seq_hidden_local = self.dropout(seq_hidden_local)

        seq_hidden_permute = seq_hidden_local
        encoded_layers = self.item_encoder(seq_hidden_permute,
                                           mask=attn_mask,
                                           src_key_padding_mask=key_padding_mask)
        sequence_output = encoded_layers

        user_emb = self.user_embeddings(user_ids).view(-1, self.hidden_size)

        gating_score_a = torch.sigmoid(torch.matmul(seq_hidden_global_a, self.gate_item.unsqueeze(0)).squeeze() +
                                       user_emb.mm(self.gate_user))
        user_seq_a = seq_hidden_global_a * gating_score_a.unsqueeze(2)
        gating_score_b = torch.sigmoid(torch.matmul(seq_hidden_global_b, self.gate_item.unsqueeze(0)).squeeze() +
                                       user_emb.mm(self.gate_user))
        user_seq_b = seq_hidden_global_b * gating_score_b.unsqueeze(2)

        user_seq_a = self.gnndrop(user_seq_a)
        user_seq_b = self.gnndrop(user_seq_b)

        hidden = torch.cat([sequence_output, user_seq_a, user_seq_b], -1)
        hidden = self.linear_transform(hidden)

        return sequence_output, hidden, user_seq_a, user_seq_b, (seq_hidden_global_a, seq_hidden_global_b), seq_mask


    def eval_stage(self, data):
        _, hidden, _, _, _, seq_mask = self.forward(data)
        hidden = self.final_att_net(seq_mask, hidden)
        return hidden


    def calculate_loss(self, data):
        targets = data[2]
        sequence_output, hidden, user_seq_a, user_seq_b, (seq_gnn_a, seq_gnn_b), seq_mask = self.forward(data)
        seq_out = self.final_att_net(seq_mask, hidden)
        seq_out = self.dropout(seq_out)
        test_item_emb = self.item_embeddings.weight[:self.item_num]
        logits = torch.matmul(seq_out, test_item_emb.transpose(0, 1))
        main_loss = self.criterion(logits, targets)

        sum_a = torch.sum(seq_gnn_a * seq_mask, 1) / torch.sum(seq_mask.float(), 1)
        sum_b = torch.sum(seq_gnn_b * seq_mask, 1) / torch.sum(seq_mask.float(), 1)

        info_hidden = torch.cat([sum_a, sum_b], 0)
        gcl_loss = self.GCL_loss(info_hidden, hidden_norm=True, temperature=0.5)

        seq_hidden_local = self.w_e(self.item_embeddings(data[1])).squeeze().unsqueeze(0)
        user_seq_a = self.w_g(user_seq_a).squeeze()
        user_seq_b = self.w_g(user_seq_b).squeeze()
        mmd_loss = self.MMD_loss(seq_hidden_local, user_seq_a) + self.MMD_loss(seq_hidden_local, user_seq_b)

        loss = main_loss + self.lam1 * gcl_loss + self.lam2 * mmd_loss
        return loss, main_loss, gcl_loss, mmd_loss
    

    def GCL_loss(self, hidden, hidden_norm=True, temperature=1.0):
        batch_size = hidden.shape[0] // 2
        LARGE_NUM = 1e9
        if hidden_norm:
            hidden = torch.nn.functional.normalize(hidden, p=2, dim=-1)
        hidden_list = torch.split(hidden, batch_size, dim=0)
        hidden1, hidden2 = hidden_list[0], hidden_list[1]

        hidden1_large = hidden1
        hidden2_large = hidden2
        labels = torch.from_numpy(np.arange(batch_size)).to(hidden.device)
        masks = torch.nn.functional.one_hot(torch.from_numpy(np.arange(batch_size)).to(hidden.device), batch_size)

        logits_aa = torch.matmul(hidden1, hidden1_large.transpose(1, 0)) / temperature
        logits_aa = logits_aa - masks * LARGE_NUM
        logits_bb = torch.matmul(hidden2, hidden2_large.transpose(1, 0)) / temperature
        logits_bb = logits_bb - masks * LARGE_NUM
        logits_ab = torch.matmul(hidden1, hidden2_large.transpose(1, 0)) / temperature
        logits_ba = torch.matmul(hidden2, hidden1_large.transpose(1, 0)) / temperature

        loss_a = torch.nn.functional.cross_entropy(torch.cat([logits_ab, logits_aa], 1), labels)
        loss_b = torch.nn.functional.cross_entropy(torch.cat([logits_ba, logits_bb], 1), labels)
        loss = (loss_a + loss_b)
        return loss
    

    # def MMD_loss(self, source, target, kernel_mul=2.0, kernel_num=5, fix_sigma=None):
    #     source = source.view(-1, self.max_seq_length)
    #     target = target.view(-1, self.max_seq_length)
    #     batch_size = int(source.size()[0])
    #     loss_all = []
    #     kernels = self.gaussian_kernel(source, target, kernel_mul=kernel_mul, kernel_num=kernel_num, fix_sigma=fix_sigma)
    #     xx = kernels[:batch_size, :batch_size]
    #     yy = kernels[batch_size:, batch_size:]
    #     xy = kernels[:batch_size, batch_size:]
    #     yx = kernels[batch_size:, :batch_size]
    #     loss = torch.mean(xx + yy - xy - yx)
    #     loss_all.append(loss)
    #     return sum(loss_all) / len(loss_all)


    def gaussian_kernel(self, source, target, kernel_mul=2.0, kernel_num=5, fix_sigma=None):
        """
        두 데이터 집합(source, target) 간의 가우시안 커널 값을 계산합니다.
        MMD Loss 계산에 사용되는 핵심 함수입니다.
        여러 개의 커널을 혼합하여 사용하는 Multi-kernel 방식을 적용합니다.

        Args:
            source (torch.Tensor): 첫 번째 분포의 샘플 텐서.
                                   Shape: (batch_size, feature_dim)
            target (torch.Tensor): 두 번째 분포의 샘플 텐서.
                                   Shape: (batch_size, feature_dim)
            kernel_mul (float): 다양한 대역폭(bandwidth)을 만들기 위한 배수.
                                기본값은 2.0입니다.
            kernel_num (int): 사용할 커널의 개수. 기본값은 5입니다.
            fix_sigma (float, optional): 대역폭(sigma) 값을 고정할 경우 사용.
                                         None일 경우, 데이터로부터 동적으로 계산됩니다.

        Returns:
            torch.Tensor: 계산된 가우시안 커널 행렬.
                          Shape: (2 * batch_size, 2 * batch_size)
        """
        # source와 target 텐서를 합쳐 전체 샘플 수를 계산합니다.
        n_samples = int(source.size()[0]) + int(target.size()[0])
        
        # 두 텐서를 concat하여 하나의 텐서로 만듭니다.
        total = torch.cat([source, target], dim=0)

        # 모든 샘플 쌍 간의 L2 거리의 제곱을 효율적으로 계산합니다.
        # total.unsqueeze(0) -> (1, n_samples, feature_dim)
        # total.unsqueeze(1) -> (n_samples, 1, feature_dim)
        # 브로드캐스팅을 통해 (n_samples, n_samples, feature_dim) 크기의 텐서 2개를 만듭니다.
        total0 = total.unsqueeze(0).expand(n_samples, n_samples, total.size(1))
        total1 = total.unsqueeze(1).expand(n_samples, n_samples, total.size(1))
        # 각 샘플 쌍의 거리 제곱을 계산하고, feature 차원에 대해 합산합니다.
        L2_distance = ((total0 - total1) ** 2).sum(2)

        # 대역폭(bandwidth) 값을 설정합니다.
        if fix_sigma:
            bandwidth = fix_sigma
        else:
            # 모든 샘플 쌍 간 거리의 평균값을 기반으로 대역폭을 추정합니다.
            bandwidth = torch.sum(L2_distance.data) / (n_samples ** 2 - n_samples)

        # Multi-kernel 방식을 위해 여러 대역폭 값을 생성합니다.
        # 기본 대역폭 값에 kernel_mul을 거듭제곱하여 곱해줍니다.
        bandwidth /= kernel_mul ** (kernel_num // 2)
        bandwidth_list = [bandwidth * (kernel_mul ** i) for i in range(kernel_num)]

        # 각 대역폭에 대해 가우시안 커널 값을 계산합니다.
        # K(x, y) = exp(-||x - y||^2 / (2 * sigma^2))
        # 여기서 bandwidth는 2 * sigma^2에 해당합니다.
        kernel_val = [torch.exp(-L2_distance / bw) for bw in bandwidth_list]
        
        # 계산된 모든 커널 값을 합산하여 최종 커널 행렬을 반환합니다.
        return sum(kernel_val)


    def MMD_loss(self, source, target, kernel_mul=2.0, kernel_num=5, fix_sigma=None):
        """
        두 분포 간의 최대 평균 불일치(Maximum Mean Discrepancy, MMD) 손실을 계산합니다.
        MMD는 두 분포가 얼마나 다른지를 측정하는 지표입니다.

        Args:
            source (torch.Tensor): 첫 번째 분포(e.g., 로컬 정보)의 샘플 텐서.
                                   Shape: (batch_size, seq_len, feature_dim) 또는 (batch_size, feature_dim)
            target (torch.Tensor): 두 번째 분포(e.g., 글로벌 정보)의 샘플 텐서.
                                   Shape: (batch_size, seq_len, feature_dim) 또는 (batch_size, feature_dim)
            kernel_mul (float): gaussian_kernel 함수로 전달될 인수.
            kernel_num (int): gaussian_kernel 함수로 전달될 인수.
            fix_sigma (float, optional): gaussian_kernel 함수로 전달될 인수.

        Returns:
            torch.Tensor: 계산된 MMD 손실 값 (스칼라 텐서).
        """
        # 입력 텐서들을 (batch_size, max_seq_length) 형태로 변환합니다.
        source = source.view(-1, self.max_seq_length)
        target = target.view(-1, self.max_seq_length)
        
        batch_size = int(source.size()[0])
        
        # gaussian_kernel 함수를 호출하여 커널 행렬을 얻습니다.
        # 이 행렬은 source와 target 샘플 간의 모든 쌍에 대한 커널 값을 포함합니다.
        kernels = self.gaussian_kernel(source, target,
                                       kernel_mul=kernel_mul,
                                       kernel_num=kernel_num,
                                       fix_sigma=fix_sigma)

        # 커널 행렬을 4개의 부분 행렬로 분할합니다.
        # XX: source 내부 샘플 간의 커널 값
        xx = kernels[:batch_size, :batch_size]
        # YY: target 내부 샘플 간의 커널 값
        yy = kernels[batch_size:, batch_size:]
        # XY: source와 target 샘플 간의 커널 값
        xy = kernels[:batch_size, batch_size:]
        # YX: target과 source 샘플 간의 커널 값
        yx = kernels[batch_size:, :batch_size]

        # MMD loss를 계산합니다.
        # MMD^2 = E[K(x, x')] + E[K(y, y')] - 2 * E[K(x, y)]
        # 위 식을 샘플 평균으로 근사한 것입니다.
        loss = torch.mean(xx + yy - xy - yx)
        
        # 계산된 손실 값을 반환합니다.
        # (loss_all 리스트는 현재 코드에서는 불필요해 보이지만 원본 로직을 유지했습니다.)
        return loss

In [33]:
model = GCL4SR(
    user_num=user_num,
    item_num=item_num,
    global_graph=global_graph,
    hidden_size=hidden_size,
    max_seq_length=max_seq_length,
    num_hidden_layers=num_hidden_layers,
    num_attention_heads=num_attention_heads,
    sample_size=sample_size,
    lam1=lam1,
    lam2=lam2
)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-5)

In [ ]:
# 위클리 미션 임을 감안하여, Validation 단계는 생략합니다.

# valid_dataset = GCL4SRData(valid_data, max_seq_length)
# valid_sampler = SequentialSampler(valid_dataset)
# valid_dataloader = DataLoader(valid_dataset, sampler=valid_sampler, batch_size=batch_size)

train_dataset = GCL4SRData(train_data, max_seq_length)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size, pin_memory=True)

test_dataset = GCL4SRData(test_data, max_seq_length)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=batch_size)

In [35]:
trainer = Trainer(model, optimizer, sample_size, hidden_size, train_matrix)

In [ ]:
for epoch in range(epochs):
    loss_list = trainer.train_step(epoch, train_dataloader)
    torch.save(model.state_dict(), checkpoint_file)

# Loss 값을 비교합니다.
grade_loss(loss_list)

/opt/anaconda3/envs/GCL4SR/lib/python3.10/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
0.16s - Expected: /opt/anaconda3/envs/GCL4SR/lib/python3.10/site-packages/debugpy/_vendored/pydevd/pydevd_attach_to_process/attach.dylib to exist.
0.13s - Expected: /opt/anaconda3/envs/GCL4SR/lib/python3.10/site-packages/debugpy/_vendored/pydevd/pydevd_attach_to_process/attach.dylib to exist.
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/opt/anaconda3/envs/GCL4SR/lib/python3.10/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/opt/anaconda3/envs/GCL4SR/lib/python3.10/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'GCL4SRData' on <module '__main__' (built-in)>


In [ ]:
trainer.model.load_state_dict(torch.load(checkpoint_file))
recall_10, recall_20, ndcg_10, ndcg_20 = trainer.eval_step(test_dataloader, test_matrix)

# Evaluation 결과를 비교합니다.
grade_eval(recall_10, recall_20, ndcg_10, ndcg_20)

WARNING: 본 교육 콘텐츠의 지식재산권은 재단법인 네이버커넥트에 귀속됩니다. 
본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다. 
다만, 비영리적 교육 및 연구활동에 한정되어 사용할 수 있으나 재단의 허락을 받아야 합니다. 
이를 위반하는 경우, 관련 법률에 따라 책임을 질 수 있습니다.